In [5]:
import requests
import urllib.request
import lxml.html as lh
import pandas as pd


In [6]:
!pip install geopy

  Using cached https://files.pythonhosted.org/packages/80/93/d384479da0ead712bdaf697a8399c13a9a89bd856ada5a27d462fb45e47b/geopy-1.20.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8b/62/26ec95a98ba64299163199e95ad1b0e34ad3f4e176e221c40245f211e425/geographiclib-1.50-py3-none-any.whl


In [7]:
from geopy.geocoders import Nominatim

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)

In [6]:
response

<Response [200]>

In [7]:
doc = lh.fromstring(response.content)

In [8]:
tr_elements = doc.xpath('//tr')

In [9]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [ ]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

In [ ]:
for c in range(1,len(tr_elements)):
    #T is our c'th row
    T=tr_elements[c]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [2]:
col

[]

#Checking for the length of the table.  Ensuring all columns are the same.

In [3]:
[len(C) for (title,C) in col]

[]

#Creating a dictionary and dataframe from the table scrapped from wiki.

In [ ]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [ ]:
df.head()

#Dropping extra Boroughs that are not assigned

In [ ]:
df = df[df.Borough != 'Not assigned']

In [ ]:
df

#Reset the index since we lost a few rows.

In [ ]:
df.reset_index()

#Checking the neighborhood values for odd variables

In [ ]:
df['Neighbourhood\n'].values

In [ ]:
df

#Dropping the \n character

In [ ]:
df['Neighbourhood\n'] = df['Neighbourhood\n'].map(lambda x: x.rstrip('\n'))
df.rename(columns={"Neighbourhood\n": "Neighbourhood"}, inplace=True)

#Replacing the neighborhood value with the Borough if the neighborhood is not assigned

In [ ]:
df.Neighbourhood\n.replace("Not assigned", df.Borough, inplace=True)

In [ ]:
df

#Loops to combine the neighborhoods together based on the Postal Code

In [ ]:
df2 = pd.DataFrame({'Postcode':df.Postcode.unique()})
df2['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=df2['Neighborhood'].apply(lambda x: ', '.join(x))
df2.dtypes

#Previewing new frame to check for accuracy

In [ ]:
df2